In [5]:
from ldap3 import Server, Connection

from config import FlaskConfig


def get_user_data(username: str, password: str) -> tuple[str, list]:
    """Get user data from LDAP Server"""

    server = Server(FlaskConfig.AD_SERVER, get_info='ALL')
    conn = Connection(
        server,
        user=f"{username}@{FlaskConfig.AD_DOMAIN}",
        password=password,
        auto_bind=True
    )
    search_parameters = {
        'search_base': FlaskConfig.AD_SEARCH_TREE,
        'search_filter': f'(sAMAccountName={username})',
        'attributes': ['cn', 'memberOf'],
    }
    conn.search(**search_parameters)
    name = conn.entries[0]['cn'] if conn.entries else ""
    groups = [
        group.split(',')[0][3:] for group in conn.entries[0]['memberOf']
    ] if conn.entries else []
    conn.unbind()
    return name, groups

In [7]:
get_user_data('savina-nn1', '088464')

LDAPBindError: automatic bind not successful - invalidCredentials

In [17]:
from ldap3 import Server, Connection, SAFE_SYNC



from ldap3 import Server, Connection, SIMPLE, SYNC, ASYNC, SUBTREE, ALL

# домен - example.com
# DNS имя сервера Active Directory
AD_SERVER = '192.168.101.253'
# Пользователь (логин) в Active Directory - нужно указать логин в AD
# в формате 'EXAMPLE\aduser' или 'aduser@example.com'
AD_USER = 'semenov-eu@orui.ru'
AD_PASSWORD = '019417'
AD_SEARCH_TREE = 'dc=orui,dc=ru'

server = Server(AD_SERVER, get_info='ALL')
conn = Connection(server, user=AD_USER, password=AD_PASSWORD, auto_bind=True)
# conn.bind()


DSA info (from DSE):
  Supported LDAP versions: 3, 2
  Naming contexts: 
    DC=orui,DC=ru
    CN=Configuration,DC=orui,DC=ru
    CN=Schema,CN=Configuration,DC=orui,DC=ru
    DC=DomainDnsZones,DC=orui,DC=ru
    DC=ForestDnsZones,DC=orui,DC=ru
  Supported controls: 
    1.2.840.113556.1.4.1338 - Verify name - Control - MICROSOFT
    1.2.840.113556.1.4.1339 - Domain scope - Control - MICROSOFT
    1.2.840.113556.1.4.1340 - Search options - Control - MICROSOFT
    1.2.840.113556.1.4.1341 - RODC DCPROMO - Control - MICROSOFT
    1.2.840.113556.1.4.1413 - Permissive modify - Control - MICROSOFT
    1.2.840.113556.1.4.1504 - Attribute scoped query - Control - MICROSOFT
    1.2.840.113556.1.4.1852 - User quota - Control - MICROSOFT
    1.2.840.113556.1.4.1907 - Server shutdown notify - Control - MICROSOFT
    1.2.840.113556.1.4.1948 - Range retrieval no error - Control - MICROSOFT
    1.2.840.113556.1.4.1974 - Server force update - Control - MICROSOFT
    1.2.840.113556.1.4.2026 - Input DN - 

In [170]:
username = 'savina-nn'

# AD_LOGIN = 'semenov-eu'
# AD_PASSWORD = '019417'
AD_LOGIN = 'savina-nn'
AD_PASSWORD = '088464'
AD_DOMAIN = "orui.ru"
AD_USER = f"{AD_LOGIN}@{AD_DOMAIN}"
AD_SERVER = '192.168.101.253'
AD_SEARCH_TREE = 'dc=orui,dc=ru'

server = Server(AD_SERVER, get_info='ALL')
conn = Connection(server, user=AD_USER, password=AD_PASSWORD, auto_bind=True)

print(conn)

def get_user_data(connection: Connection, user_name: str) -> tuple[str|None, list]:
    search_parameters = {
        'search_base': AD_SEARCH_TREE,
        'search_filter': f'(sAMAccountName={user_name})',
        'attributes': ['cn', 'memberOf'],
    }
    connection.search(**search_parameters)
    if conn.entries:
        name = conn.entries[0]['cn']
        groups = [group.split(',')[0][3:] for group in conn.entries[0]['memberOf']]
        return name, groups
    else:
        return None, []


name, groups = get_user_data(conn, username)


print(username, name, groups)

ldap://192.168.101.253:389 - cleartext - user: savina-nn@orui.ru - not lazy - bound - open - <local: 192.168.100.131:65133 - remote: 192.168.101.253:389> - tls not started - listening - SyncStrategy - internal decoder
savina-nn Савина Наталья Николаевна ['Апекс-ВУЗ - Методист', 'Сотрудники', 'Кафедра информационных технологий в деятельности ОВД']


In [132]:
from dataclasses import dataclass


@dataclass
class AdUser:

    username: str
    conn: Connection

    def search_data(self):
        searchParameters = {
        'search_base': 'dc=orui,dc=ru',
        'search_filter': f'(sAMAccountName={self.username})',
        # 'search_scope': 'SUBTREE',
        'attributes': ['cn', 'memberOf'],
        # 'paged_size': 5,
        }
        self.conn.search(**searchParameters)
        return self.conn.entries

    def get_groups(self):
        conn.search(**self.searchParameters)
        for entry in conn.entries:
            print(entry['memberOf'])


# searchParameters = {
#     'search_base': 'dc=orui,dc=ru',
#     'search_filter': '(sAMAccountName=semenov-eu)',
#     # 'search_scope': 'SUBTREE',
#     'attributes': ['cn', 'memberOf'],
#     # 'paged_size': 5,
# }

# conn.search(**searchParameters)
# for entry in conn.entries:
#     print(entry['memberOf'])

['CN=Апекс-ВУЗ - Полный доступ,CN=Users,DC=orui,DC=ru', 'CN=Апекс-ВУЗ - Преподаватель,CN=Users,DC=orui,DC=ru', 'CN=Сотрудники,OU=GROUP,DC=orui,DC=ru', 'CN=Кафедра информационных технологий в деятельности ОВД,OU=GROUP,DC=orui,DC=ru']


In [18]:
print(server.schema)

DSA Schema from: CN=Aggregate,CN=Schema,CN=Configuration,DC=orui,DC=ru
  Attribute types:{'msExchMessageHygieneBlockedDomain': Attribute type: 1.2.840.113556.1.4.7000.102.50621
  Short name: msExchMessageHygieneBlockedDomain
  Single value: False
  Syntax: 1.3.6.1.4.1.1466.115.121.1.15 [('1.3.6.1.4.1.1466.115.121.1.15', 'LDAP_SYNTAX', 'Directory String', 'RFC4517')]
  Optional in: msExchMessageHygieneSenderFilterConfig
, 'sIDHistory': Attribute type: 1.2.840.113556.1.4.609
  Short name: sIDHistory
  Single value: False
  Syntax: 1.3.6.1.4.1.1466.115.121.1.40 [('1.3.6.1.4.1.1466.115.121.1.40', 'LDAP_SYNTAX', 'Octet String', 'RFC4517')]
  Optional in: securityPrincipal
, 'revision': Attribute type: 1.2.840.113556.1.4.145
  Short name: revision
  Single value: True
  Syntax: 1.3.6.1.4.1.1466.115.121.1.27 [('1.3.6.1.4.1.1466.115.121.1.27', 'LDAP_SYNTAX', 'Integer', 'RFC4517')]
  Optional in: samDomainBase, top
, 'msTSMaxIdleTime': Attribute type: 1.2.840.113556.1.4.1983
  Short name: msTSM

In [111]:
data = conn.extend.standard.who_am_i()

In [115]:
conn.

'SIMPLE'

In [21]:
conn.extend.standard.who_am_i()

'u:ORUI\\semenov-eu'

In [22]:
conn.start_tls()

LDAPStartTLSError: startTLS failed - unavailable